In [8]:
import json

In [4]:
import requests
res = requests.get('http://localhost:9200')

In [5]:
# connect to es cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [70]:
# dbl metaphone index settings
setting_dbl = {
  "settings": {
    "analysis": {
      "filter": {
        "double_metaphone": {
          "type": "phonetic",
          "encoder": "double_metaphone"
        }
      },
      "analyzer": {
        "double_metaphone": {
          "tokenizer": "standard",
          "filter": "double_metaphone"
        }
      }
    }
  },
  "mappings": {
    "people": {
      "properties": {
        "name": {
          "type": "text",
          "fields": {
            "phonetic": {
              "type": "text",
              "analyzer": "double_metaphone"
            }
          }
        }
      }
    }
  }
}

In [71]:
# create dbl metaphone index
es.indices.create(index='dbl_test', ignore=400, body=setting_dbl)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'dbl_test'}

In [72]:
# Putting in phonetically similar names
name1 = {'name': 'John Smith'}
name2 = {'name': 'Jonnie Smythe'}

es.index(index='dbl_test', doc_type='people', id =1, body=name1) 
es.index(index='dbl_test', doc_type='people', id =2, body=name2) 

{'_index': 'dbl_test',
 '_type': 'people',
 '_id': '2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [74]:
# Search names phonetically
es.search(index='dbl_test', body={'query': 
                             {'match': 
                              {'name.phonetic':
                               {'query': 'Jahnnie Smeeth',
                                'operator': 'and'
                               }}}})

{'took': 11,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 2,
  'max_score': 0.9205827,
  'hits': [{'_index': 'dbl_test',
    '_type': 'people',
    '_id': '2',
    '_score': 0.9205827,
    '_source': {'name': 'Jonnie Smythe'}},
   {'_index': 'dbl_test',
    '_type': 'people',
    '_id': '1',
    '_score': 0.9205827,
    '_source': {'name': 'John Smith'}}]}}

### Different ways to use ES 

In [ ]:
# Way to pump in data using API calls
# pump in Star Wars data
r = requests.get('http://localhost:9200')
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='dbl', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1

# pump in Star Wars data
r = requests.get('http://localhost:9200')
i = 18
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='dbl', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1

In [44]:
# Search via prefix
es.search(index="dbl", body={"query": {"prefix" : { "name" : "lu" }}})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 2,
  'max_score': 1.0,
  'hits': [{'_index': 'dbl',
    '_type': 'people',
    '_id': '1',
    '_score': 1.0,
    '_source': {'name': 'Luke Skywalker',
     'height': '172',
     'mass': '77',
     'hair_color': 'blond',
     'skin_color': 'fair',
     'eye_color': 'blue',
     'birth_year': '19BBY',
     'gender': 'male',
     'homeworld': 'https://swapi.co/api/planets/1/',
     'films': ['https://swapi.co/api/films/2/',
      'https://swapi.co/api/films/6/',
      'https://swapi.co/api/films/3/',
      'https://swapi.co/api/films/1/',
      'https://swapi.co/api/films/7/'],
     'species': ['https://swapi.co/api/species/1/'],
     'vehicles': ['https://swapi.co/api/vehicles/14/',
      'https://swapi.co/api/vehicles/30/'],
     'starships': ['https://swapi.co/api/starships/12/',
      'https://swapi.co/api/starships/22/'],
     'created': '2014-12-09T13:50:51.6440